![Erudio logo](img/erudio-logo-small.png)
---
![Sklearn logo](img/scikit-learn-logo-small.png)

# Intermediate Machine Learning with scikit-learn

## Clustering

For an introduction to clustering algorithms, let us look at another visualization copied from the [scikit-learn documentation](http://scikit-learn.org/stable/modules/clustering.html). This is a visualization of several synthetic distributions in two dimensions to allow visualization.  We can  see that some distributions that do not have convex differentiation throw off some clutering algorithms.  Of course, given the low dimensionality, these examples are somewhat toys.

In [ ]:
from src.setup import *

In [ ]:
%matplotlib inline
%run src/plot_cluster_comparison.py

## Higher Dimensions

For examples with greater dimensionality, we will look first at a couple similar datasets in the scikit-learn samples.  A small-ish set of 1460 houses sold in Ames Iowa from 2006 to 2010. A different, larger sample dataset of 20k houses in California, derived from the 1990 census, is also available.  The features contained in the two datasets have both overlaps (albeit with different names for corresponding features) and differences.

Although normally clustering is used identify patterns where no *a priori* classes or target values are available, by using these datasets, we can test a certain intuition I have.  That is, my hunch would be that the way houses cluster along many feature dimensions will correspond to the clustering or distribution in the single price target.

My hypothesis can well be wrong without meaning that clustering is meaningless.  The semantic sense of the clusters may simply be something other than price—perhaps some other implicit feature we have a good name for, but also perhaps something with no obvious English word corresponding to the essence of the classes.  Let us explore this.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

warnings.simplefilter("ignore")

In [ ]:
from sklearn.datasets import fetch_openml

ames = fetch_openml(name="house_prices", as_frame=True)
print(ames.DESCR)

In [ ]:
ames_df = pd.DataFrame(ames.data, columns=ames.feature_names)
ames_df["SalePrice"] = ames.target
ames_df

Because of the nature of the clustering techniques we want to apply, we are only interested in numerical variables.

In [ ]:
ames_numerical_df = ames_df.select_dtypes(include="number")
ames_numerical_df

Of the 79 variables, 37 are numeric. The first variable is only the incremental
identifier `Id`.

From these 37 variables, we selected only those that have the highest correlation with the target variable `SalePrice` and discarded some variables that have basically the same meaning. For example, `GarageCars` outweighs `GarageArea`, the capacity of the garage in number of cars or the area of the garage.

* `OverallQual`: Ratings of the overall quality on a 1-10 scale.
* `GrLivArea`: The above ground living area in square feet.
* `GarageCars`: Capacity of the garage in terms of the number of cars it can accommodate.
* `TotalBsmtSF`: the total area of the basement in square feet.
* `FullBath`: The number of full bathrooms. More bathrooms increase prices.
* `YearBuilt`: The year the house was originally constructed. Age is a key price determinant.


`SalePrice`: The sale price of the house, which is the target variable to be predicted in regression problems.

In [ ]:
ames_cluster_df = ames_numerical_df.set_index("Id")[
    [
        "OverallQual",
        "GrLivArea",
        "GarageCars",
        "TotalBsmtSF",
        "FullBath",
        "YearBuilt",
        "SalePrice",
    ]
]
ames_cluster_df

In [ ]:
plt.figure(figsize=(10, 2))
sns.heatmap(
    ames_cluster_df.corr(),
    vmin=-1,
    vmax=1,
    annot=True,
    cmap="BrBG",
);

### Distances in Parametric Space

This is mostly for a later lesson, but many clustering algorithms are sensitive to the distances between points in a parameteric space in a unit-dependent way.  A later lesson on feature engineering will look at scaling data in more detail.  For some of the examples in this lesson, scaling will make little difference; however, for a few of them the absolute units of features becomes important.

Therefore, in the below examples, you will see line like this in the cells that fit models:

```python
X = StandardScaler().fit_transform(the_data)
```

In [ ]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(ames_cluster_df.drop(columns="SalePrice").values)

In [ ]:
X

## Cluster into N Classes

For the next few cells, we will use `SpectralClustering` to group our dataset into four synthetic classes.  Informally, we will call these classes "low", "mid_low", "mid_high", and "high" corresponding to our hypothesis that price is a good proxy for these classes.  From the documentation, the algorithm is described as follows:

> `SpectralClustering` does a low-dimension embedding of the affinity matrix between samples, followed by a KMeans in the low dimensional space. It is especially efficient if the affinity matrix is sparse and the [`pyamg`](https://github.com/pyamg/pyamg) module is installed. `SpectralClustering` requires the number of clusters to be specified. It works well for a small number of clusters but is not advised when using many clusters.

In [ ]:
from sklearn import cluster

spectral = cluster.SpectralClustering(
    n_clusters=4, eigen_solver="arpack", affinity="nearest_neighbors"
)

spectral.fit(X)

After fitting the data, what we get is labels for each sample.  These are simply integers in `range(n_clusters)`, with no inherent semantic meaning.  I.e. 0, 1, 2, or 3 in this case.

In [ ]:
print(spectral.labels_[:100])

After we do the clustering, let us group the dataset by the cluster labels.  We will enhance the Pandas DataFrame that holds the data by adding columns for the derived category and the known price.  We also give hopefully more meaningful names to the clusters.  We only display some of the features to narrow discussion.

In [ ]:
price_cats = ['low', 'mid_low', 'mid_high', 'high']
ames_cluster_df['category'] = spectral.labels_
ames_clusters = ames_cluster_df.groupby('category').mean().sort_values('SalePrice')
ames_cluster_df.category = ames_cluster_df.category.map({k:v for k,v in zip(ames_clusters.index, price_cats)})
ames_clusters.index = price_cats
cols = ['SalePrice'] + list(ames_clusters.columns[:-1])
ames_clusters = ames_clusters[cols]

In [ ]:
ames_clusters

At first brush, it looks like price is pretty well differentiated by cluster.  Remember that price did not participate in the clustering model; so inasmuch as they correspond, it confirms our working hypothesis.

Other columns have interstingly different patterns.  Crime rate is high for the "low" category houses, then drops to almost none for "mid_low", "mid_high", and "high."  In contrast, the number of rooms varies little by category.  Distance to employment centers is low and similar for the first three categories of houses, but much larger for the "high" category houses.  Age of house shows a similar, but less dramatic, pattern as distance.

Let us try to get a handle on who well price is genuinely differentiated by category.

In [ ]:
clusters = (
    ames_cluster_df.groupby("category")
    .SalePrice.agg(["count", "mean", "median", "std"])
    .sort_values("mean")
)
clusters.index = ["low", "mid_low", "mid_high", "high"]
clusters

In a visual representation of the pattern we see *pretty good* but not overwhelming differentiation. One category of houses show a wide range of prices, while other categories tend to be more narrowly centered.

In [ ]:
sns.displot(
    data=ames_cluster_df,
    x="SalePrice",
    hue="category",
    kind="kde",
    rug=True,
    height=4,
    aspect=2.5,
).set(title="Distribution of clusters on Sale Price")

Given our fairly successful confirmation of our hypothesis in regard to 2000s Ames housing prices relative to independent features, let us try the same exercise with the much larger 1990s California housing prices dataset.

In [ ]:
from sklearn import datasets

california = datasets.fetch_california_housing()
ca_df = pd.DataFrame(california.data, columns=california.feature_names)
ca_df['SalePrice'] = california.target
ca_df.head()

In [ ]:
%%time
# Takes a little while to cluster 20k observations
from sklearn import cluster
spectral = cluster.SpectralClustering(
    n_clusters=4,
    eigen_solver='arpack',
    affinity="nearest_neighbors"
)

X = StandardScaler().fit_transform(california.data)
spectral.fit(X)

In [ ]:
# Let us do minor Pandas manipulation to view clusters in an intuitive way
ca_df['category'] = spectral.labels_
ca_clusters = ca_df.groupby('category').mean().sort_values('SalePrice')
ca_clusters.index = ['low', 'mid_low', 'mid_high', 'high']
ca_clusters

The clustering here seems to be mostly independent of price.  The only things there really seem differentiated in the cluster categories are age of house (young houses at extremes of price scale), number of rooms, and especially size of the cities where the houses are located.  Even the north/south or east/west distinctions I would expect in California houses are not strongly exposed by this technique.

We can look more specifically at the price distributions in groups, but it shows nothing relevant (nor does the more detailed graph; other than a somewhat bimodel distribution for all categories).

In [ ]:
(
    ca_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std"])
    .sort_values("median")
)

In [ ]:
sns.displot(
    data=ca_df, x="SalePrice", hue="category", kind="kde", rug=True, height=4, aspect=2.5
).set(title="Distribution of clusters on Sale Price");

## Cluster into an Unknown Number of Categories

In the first pass, we chose the number of clusters we expect in the data manually.  Many models require an `n_clusters` argument to select this.  However, some of the models determine the number of clusters from the data itself.  That is not uniformly better, but may let the "data speak" better in many cases.

Let us try our same analysis using the very good `DBSCAN` algorithm (Density-based spatial clustering of applications with noise).  This model is often the go-to choice for data scientists:

> The [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples).

A result of this algorithm is that the number of clusters detected depends strongly on an epsilon hyperparameter that measures the acceptable distance to extend the region of a "core" and to reach non-core samples.  This makes scaling important to get consistent units among features, but also makes the choice of epsilon have a strong effect on number of identified clusters.  Notwithstanding the simple illustration below, the "core" region can be extended in any shape by identifying additional core points along any path.

"Noise points" in DBSCAN are given the class label -1.

<img src="img/DBSCAN.png">

In [ ]:
import numpy as np

X = StandardScaler().fit_transform(
    ames_cluster_df.drop(columns=["SalePrice", "category"]).values
)

for epsilon in np.arange(0.1, 1.8, 0.1):
    db = cluster.DBSCAN(eps=epsilon)
    db.fit(X)
    labels = np.unique(db.labels_)
    print("Epsilon=%.1f; labels=[%d .. %d]" % (epsilon, labels[0], labels[-1]))

In [ ]:
X = StandardScaler().fit_transform(california.data)

for epsilon in np.arange(0.1, 1.8, 0.1):
    db = cluster.DBSCAN(eps=epsilon)
    db.fit(X)
    labels = np.unique(db.labels_)
    print("Epsilon=%.1f; labels=[%d .. %d]" % (epsilon, labels[0], labels[-1]))

### Evaluating Price Hypothesis with DBSCAN

As you can see, by choosing different epsilon values, we can get wildly different numbers of clusters using DBSCAN.  This is not a bad thing, and being able to choose the granularity of clustering can be desirable.  However, on closer examination, we notice that  with various epsilon values, we still wind up either with most data samples in category -1 for noise, or in the overly generic category 0.  It may be that further hyperparameter tuning could improve this, but we will not do that for the California dataset.

In [ ]:
X = StandardScaler().fit_transform(california.data)
ca_df["category"] = cluster.DBSCAN(eps=0.5).fit(X).labels_

(
    ca_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std", "count"])
    .sort_values("count", ascending=False)
    .head()
)

Increasing `min_samples` from the default 5 to 20 *does* improve matters somewhat. Other hyperparameter values might continue to improve this.

In [ ]:
X = StandardScaler().fit_transform(california.data)
ca_df["category"] = cluster.DBSCAN(eps=0.5, min_samples=20).fit(X).labels_

(
    ca_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std", "count"])
    .sort_values("count", ascending=False)
    .head()
)

With the Ames dataset we can get much less problem with noise and dominance of cluster 0.  Of course, we already had reasonable confirmation of our initial price-based clustering with spectral clustering.

In [ ]:
X = StandardScaler().fit_transform(
    ames_cluster_df.drop(columns=["SalePrice", "category"]).values
)
ames_cluster_df["category"] = cluster.DBSCAN(eps=1.3).fit(X).labels_

(
    ames_cluster_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std", "count"])
    .sort_values("count", ascending=False)
)

In [ ]:
g = sns.displot(
    data=ames_cluster_df[ames_cluster_df.category != -1],
    x="SalePrice",
    hue="category",
    kind="kde",
    rug=True,
    height=4,
    aspect=2.5,
).set(title="Distribution of clusters on Sale Price")

I believe our conclusion at this point should be that there are several price-centric categories of houses in the Ames dataset, but for other classes that are clustered, different factors take precedent.  Four or five of the 8 categories have steep Gaussian slopes, that rest are distributed in ways suggesting price is not crucial.

## HDBSCAN: A Better DBSCAN?

A colleague of mine advised me to try out an improved variant of DBSCAN called [HDBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html). This algorithm was adopted in scikit-learn version 1.3.  The basic idea in HDBSCAN is that the algorithm will dynamically adjust the epsilon value in regions of varying density.  From the [Campello, Moulavi, and Sander](http://link.springer.com/chapter/10.1007%2F978-3-642-37456-2_14) research paper that introduced it:

> We propose a theoretically and practically improved density-based, hierarchical clustering method, providing a clustering hierarchy from which a simplified tree of significant clusters can be constructed. For obtaining a “flat” partition consisting of only the most significant clusters (possibly corresponding to different density thresholds), we propose a novel cluster stability measure, formalize the problem of maximizing the overall stability of selected clusters, and formulate an algorithm that computes an optimal solution to this problem. We demonstrate that our approach outperforms the current, state-of-the-art, density-based clustering methods on a wide variety of real world data.

In [ ]:
X = StandardScaler().fit_transform(
    ames_cluster_df.drop(columns=["SalePrice", "category"]).values
)
hdb = cluster.HDBSCAN(min_cluster_size=20)
hdb.fit(X)
labels = np.unique(hdb.labels_)
print("Labels=[%d .. %d]" % (labels[0], labels[-1]))

In [ ]:
ames_cluster_df["category"] = hdb.labels_
(
    ames_cluster_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std", "count"])
    .sort_values("count", ascending=False)
    .head()
)

In [ ]:
g = sns.displot(
    data=ames_cluster_df[ames_cluster_df.category != -1],
    x="SalePrice",
    hue="category",
    kind="kde",
    rug=True,
    height=4,
    aspect=2.5,
).set(title="Distribution of clusters on Sale Price")

On the relatively confirmatory Ames housing dataset, HDBSCAN produces sharper price peaks than other clustering approaches we have looked at. However, as with other techniques, it appears to identify several price centers *and also* other clusters whose identity is not chiefly price-focused.

HDBSCAN both gave us automated determination of number of clusters and avoided the need for manually tweaking epsilon. But the Ames dataset has been relatively easy, let us try against the California housing dataset.

In [ ]:
X = StandardScaler().fit_transform(california.data)

hdb = cluster.HDBSCAN(min_cluster_size=20)
hdb.fit(X)
labels = np.unique(hdb.labels_)
print("Labels=[%d .. %d]" % (labels[0], labels[-1]))

In [ ]:
ca_df["category"] = hdb.labels_
(
    ca_df.groupby("category")
    .SalePrice.agg(["mean", "median", "std", "count"])
    .sort_values("count", ascending=False)
    .head()
)

This approach is still relatively unsucessessful.  At least it does not identify the assumed price-basis of the clusters.  It may be time to reject our hypothesis relative to this dataset.

In [ ]:
sns.displot(
    data=ca_df, x="SalePrice", hue="category", kind="kde", rug=True, height=4, aspect=2.5
).set(title="Distribution of clusters on Sale Price");

## Evaluating Clustering

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/clustering.htm):

> Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

The problem, of course, is that in **unsupervised** learning, you are more-or-less assured of not having a "ground truth" since you are trying to tease that out of the data.  In the examples above, we partially faked it by having a hypothesis about ground truth; but that hypothesis seemed supportable in one dataset and unsupportable in a different related dataset[^1].


[^1]: <small><i>It is accurate to say that we have failed to validate our hypothesis, not that we have proven it false.  It might still be that some additional clustering model we have not tried—or some hyperparameters to those we tried—would produce price-linked clusters in the California dataset of similar strength to those the Boston dataset readily showed.</i></small>


One kind of validation that is possible to perform is to see whether different clustering algorithms produce similar classes, and assign observations to the same class as each other.  If there is agreement, that lends *some* extra weight to the idea that the clusters represent an underlying phenomenon of the data.

The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.  This method is symmetric and independent of the particular labels an algorithm uses.  It depends only on collections of samples in the same class under one regime correspondng to the same collection under another regime.  *Adjusted mutual information score* is similar to *adjusted Rand score*.

In [ ]:
# Fit several clustering models that take n_clusters
from sklearn import cluster

X = StandardScaler().fit_transform(
    ames_cluster_df.drop(columns=["SalePrice", "category"]).values
)

models = [
    cluster.SpectralClustering(n_clusters=4),
    cluster.KMeans(n_clusters=4),
    cluster.MiniBatchKMeans(n_clusters=4),
    cluster.AgglomerativeClustering(n_clusters=4),
]

for model in models:
    model.fit(X)
    print(model.__class__.__name__)
    print("\tFirst 5 labels", model.labels_[:10])
    print("\t", len(model.labels_))

In the example above, the clustering models selected assigned different classes to the first five samples. However, the particular number given to that class might be any of the four possible values (depending even on random seeds).  The label numbers are not interesting, only the samples assigned to them.

In [ ]:
# ... then measure their pairwise similarity
from itertools import combinations

from sklearn import metrics

for clust1, clust2 in combinations(models, 2):
    print(clust1.__class__.__name__, "versus", clust2.__class__.__name__)
    print("\tRand score:", metrics.adjusted_rand_score(clust1.labels_, clust2.labels_))
    print(
        "\tMutual info:",
        metrics.adjusted_mutual_info_score(clust1.labels_, clust2.labels_),
    )

Both measures made range between zero and one (hence minimum can be negative).  As one would expect, related
methods `KMeans`, `MiniBatchKMeans` and `AgglomerativeClustering` are somehow
similar.  Other pairs are less similar; certainly they are much more similar
than e.g. random choices of labels, which would produce scores of zero (the
*adjusted* part discounts for expected random intersections).

## Next Lesson

**Feature Engineering**: In the current lessson we looked at several clustering models.  Besides their underlying algorithmic differences, a key difference among them is whether they choose a number of clusters based on the data itself or produce a specified number of classes. 

In this lesson and some previous one, we have done some feature engineering in passing. In the next lessons we will address dimensionality reduction, feature engineering, and feature selection, in more specific detail.

<a href="SKLearn-07_Decomposition.ipynb"><img src="img/open-notebook.png" align="left"/></a>


---

Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors